In [14]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.special import boxcox1p
import pickle
from os import path
import statsmodels.api as sm
import warnings

warnings.filterwarnings("ignore")

# data Visualization
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import plotly.express as px 
from plotnine import ggplot, aes, geom_line, geom_point, facet_wrap, theme
import plotly.graph_objects as go
# Machine learning algorithms and model evaluation
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import bayesian_search_forecaster
from lightgbm import LGBMRegressor
# Encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


1. Data Preprocessing:
Load the datasets


In [15]:
# Load the three datasets
average_hours = pd.read_csv('Resources/average-usual-weekly-hours-worked-women-15-years-and-older.csv')
employment_ratio = pd.read_csv("Resources/female-employment-to-population-ratio.csv")
wage_gap = pd.read_csv("Resources/gender-wage-gap-oecd.csv")
percapita_labor = pd.read_csv("Resources/female-labor-force-participation-rates-by-national-per-capita-income.csv")
school_years = pd.read_csv("Resources/mean-years-of-schooling-female.csv")
maternity_leave = pd.read_csv("Resources/paid-leave-at-least-14-weeks-mothers.csv")
labor_sector = pd.read_csv("Resources/share-of-female-workers-by-sector.csv")


In [16]:
print(f'Shape of average_hours DataFrame: {average_hours.shape}')
print(f'Shape of employment_ratio DataFrame: {employment_ratio.shape}')
print(f'Shape of wage_gap DataFrame: {wage_gap.shape}')
print(f'Shape of percapita_labor DataFrame: {percapita_labor.shape}')
print(f'Shape of school_years DataFrame: {school_years.shape}')
print(f'Shape of maternity_leave DataFrame: {maternity_leave.shape}')
print(f'Shape of labor_sector DataFrame: {labor_sector.shape}')

Shape of average_hours DataFrame: (971, 4)
Shape of employment_ratio DataFrame: (3919, 4)
Shape of wage_gap DataFrame: (636, 4)
Shape of percapita_labor DataFrame: (7185, 6)
Shape of school_years DataFrame: (1502, 4)
Shape of maternity_leave DataFrame: (10206, 4)
Shape of labor_sector DataFrame: (2781, 6)


1. Data Preprocessing: Rename Columns to make it shorter

In [17]:
average_hours = average_hours.rename(columns={'Average weekly hours worked (women, 15+) (OECD Labor Force Statistics (2017))': 'Avg_Hours_Worked'})
employment_ratio = employment_ratio.rename(columns={'Employment to population ratio, 15+, female (%) (national estimate)': 'Emp_Pop_Ratio'})
wage_gap = wage_gap.rename(columns={'Gender wage gap (OECD 2017)': 'Gender_Wage_Gap'})
percapita_labor = percapita_labor.rename(columns={'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)': 'Labor_Force'})
school_years = school_years.rename(columns={'Mean years of schooling (ISCED 1 or higher), population 25+ years, female': 'School_Years_Mean'})
maternity_leave = maternity_leave.rename(columns={'Paid leave of at least 14 weeks available to mothers (1=yes; 0=no)': 'Paid_Leave'})
labor_sector = labor_sector.rename(columns={'Female share of employment in agriculture (%)': 'Argiculture','Female share of employment in industry (%)': 'Industry','Female share of employment in services (%': 'Services'})


In [18]:
#Drop Null Values for female-employment-to-population-ratio.csv
employment_ratio.dropna(inplace=True)
print(employment_ratio.isnull().sum())

Entity           0
Code             0
Year             0
Emp_Pop_Ratio    0
dtype: int64


In [19]:
#Drop Null Values for average-usual-weekly-hours-worked-women-15-years-and-older.csv
average_hours.dropna(inplace=True)
print(average_hours.isnull().sum())

Entity              0
Code                0
Year                0
Avg_Hours_Worked    0
dtype: int64


In [27]:
percapita_labor.dropna(inplace=True)
print(percapita_labor.isnull().sum())

Entity                                                 0
Code                                                   0
Year                                                   0
Labor_Force                                            0
GDP per capita, PPP (constant 2017 international $)    0
Continent                                              0
dtype: int64


In [28]:
# Merge the datasets
merged_df= average_hours.merge(employment_ratio, on=['Entity', 'Code', 'Year'], how='outer')
merged_df= merged_df .merge(wage_gap, on=['Entity', 'Code', 'Year'], how='outer')
merged_df= merged_df .merge(percapita_labor, on=['Entity', 'Code', 'Year'], how='outer')
merged_df= merged_df .merge(school_years, on=['Entity', 'Code', 'Year'], how='outer')
merged_df= merged_df .merge(maternity_leave, on=['Entity', 'Code', 'Year'], how='outer')
merged_df= merged_df .merge(labor_sector, on=['Entity', 'Code', 'Year'], how='outer')

# Check the result
print(merged_df.shape)

(10497, 14)


In [29]:
#Drop colum "Continent"
merged_df = merged_df.drop("Continent", axis=1)

1. Data Preprocessing: Check datatypes for each of the dataset columns.

In [30]:
# Check for null values
print(merged_df.isnull().sum())

Entity                                                     0
Code                                                      29
Year                                                       0
Avg_Hours_Worked                                        9609
Emp_Pop_Ratio                                           6813
Gender_Wage_Gap                                         9861
Labor_Force                                            10322
GDP per capita, PPP (constant 2017 international $)    10322
School_Years_Mean                                       8995
Paid_Leave                                               291
Argiculture                                             7772
Industry                                                7750
Female share of employment in services (%)              7752
dtype: int64


In [23]:
# Check for duplicates
duplicates = merged_df.duplicated().sum()
print(f"Number of duplicates: {duplicates}")

Number of duplicates: 0


In [24]:
# Check the data types of each column
print(merged_df.dtypes)


Entity                                                  object
Code                                                    object
Year                                                     int64
Avg_Hours_Worked                                       float64
Emp_Pop_Ratio                                          float64
Gender_Wage_Gap                                        float64
Labor_Force                                            float64
GDP per capita, PPP (constant 2017 international $)    float64
School_Years_Mean                                      float64
Paid_Leave                                             float64
Argiculture                                            float64
Industry                                               float64
Female share of employment in services (%)             float64
dtype: object


In [25]:
# Perform some basic EDA
print(merged_df.describe())


               Year  Avg_Hours_Worked  Emp_Pop_Ratio  Gender_Wage_Gap  \
count  11400.000000        888.000000    3684.000000       636.000000   
mean    1997.455263         35.485752      44.682533        19.385535   
std       15.469445          4.047832      14.474665        10.053530   
min     1960.000000         24.038079       2.500000         0.000000   
25%     1984.000000         32.523503      36.704750        13.200000   
50%     1998.000000         35.616393      46.044000        17.900000   
75%     2011.000000         38.548941      53.077250        23.400000   
max     2023.000000         44.813507      94.500000        52.800000   

       Labor_Force  GDP per capita, PPP (constant 2017 international $)  \
count  6431.000000                                        6562.000000     
mean     49.584312                                       18307.128922     
std      15.370705                                       20344.959620     
min       4.948000                        

In [26]:
# Check for outliers in numerical columns
numerical_cols = merged_df.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    Q1 = merged_df[col].quantile(0.25)
    Q3 = merged_df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = merged_df[(merged_df[col] < Q1 - 1.5 * IQR) | (merged_df[col] > Q3 + 1.5 * IQR)]
    print(f"Outliers in column {col}: {outliers.shape[0]}")


Outliers in column Year: 0
Outliers in column Avg_Hours_Worked: 0
Outliers in column Emp_Pop_Ratio: 152
Outliers in column Gender_Wage_Gap: 44
Outliers in column Labor_Force: 210
Outliers in column GDP per capita, PPP (constant 2017 international $): 269
Outliers in column School_Years_Mean: 0
Outliers in column Paid_Leave: 0
Outliers in column Argiculture: 0
Outliers in column Industry: 61
Outliers in column Female share of employment in services (%): 239
